# Predicting Students' Academic Success


*By Chanon Mallikamas, Sylvia Ying, Xiaolei Lin, and Erin Kang*

## FYI

how to italicize: 
*hi*

how to bold: 
**text** 

how to decrease text size: 
increase number of # used 

example, 
with three ### 
### Title
with five ##### 
##### Title

## Introduction

From Rubric: 
*Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
Clearly state the question you will try to answer with your project
Identify and describe the dataset that will be used to answer the question*

OUR CHARACTER: 
gender (female), age of enrollment (18), sem1 grade (8), sem2 grade (13), course (fixed! - 12)

In [1]:
#load necessary packages 
library(tidyverse)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [2]:
#set seed for the analysis to be reproducible
set.seed(1)

In [3]:
#load data into R
#use head() since data is too large
data_set<-read_csv("https://raw.githubusercontent.com/ekang27kc/dsci100-2023-T2-Group01-project/main/dataset.csv")
head(data_set)

Rows: 4424 Columns: 35
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): Target
dbl (34): Marital status, Application mode, Application order, Course, Dayti...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,Mother's occupation,⋯,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,8,5,2,1,1,1,13,10,6,⋯,0,0,0,0,0.00000,0,10.8,1.4,1.74,Dropout
1,6,1,11,1,1,1,1,3,4,⋯,0,6,6,6,13.66667,0,13.9,-0.3,0.79,Graduate
1,1,5,5,1,1,1,22,27,10,⋯,0,6,0,0,0.00000,0,10.8,1.4,1.74,Dropout
1,8,2,15,1,1,1,23,27,6,⋯,0,6,10,5,12.40000,0,9.4,-0.8,-3.12,Graduate
2,12,1,3,0,1,1,22,28,10,⋯,0,6,6,6,13.00000,0,13.9,-0.3,0.79,Graduate
2,12,1,17,0,12,1,22,27,10,⋯,0,5,17,5,11.50000,5,16.2,0.3,-0.92,Graduate


# Methods

Explain how you will conduct either your data analysis and which variables/columns you will use. 
For each variable think: is this a useful variable for prediction?
Describe at least one way that you will visualize the results

Goal: Create a training set to classify whether or not a female undergraduate student in nursing who enrolled to university at age 18 and gained a 60% and 70% grade in their first and second semester, respectively, in their first year.

#### Step One: Tidy & Wrangling Data

Using the **select()** function, select only the columns needed to answer the predictive question, which are Gender, Age at enrollment, Curricular units 1st sem (grade), and Curricular units 2nd sem (grade).

Using the **filter()** function, select only the observations/rows of students who are in nursing (course is nursing). 
Using the same function, filter out rows with "Enrolled" as the Target (the unwanted label for analysis). 

Using **!is.na**, remove all na values in each of the chosen columns.

Since Target (for Graduate or Dropout) is the chosen label, using the **as_factor()** function, convert Target to a factor variable from a character variable.  


##### Step Two: Training Data + Selecting Perameter Value

Using the inital_split() function, split the data set into a training and testing data set. 

Train the training data set by: 

1. Using the **recipe()** function, creating a recipe specifying the label and predictors' columns and standardizing the data set to ensure all data are in a comparable scale that contribute equally to k-nearest neighbor calculations. 
2. Using the **nearest_neighbor()** function, specify the model (k-nearest neighbor) with tuning on the number of neighbors 
3. Using the **vfold_cv** funciton, perform a 5-fold validation to evaluate the accuracy of the classifier of each k value
4. Using the **workflow()** function, create a workflow analysis with the created recipe, model specification, cross-validation model as well as with 10 k values and the tune_grid function to fit the model for each k value. 
5. Using the **collect_metrics** function, collect the means (estimated accuracies) of classifiers with different k values
6. Using the **filter()** function, filter rows/observations with "accuracy" for .metric to only consider the means (estimated accuracies) of the classifiers.  
7. Plot a mean vs neighbor number graph to visually predict the optimum k value for training the training set. 

##### Step Three: Predicting Target (Label)

